In [ ]:
#%pip install --upgrade --quiet  langchain-openai
#%pip install python-dotenv --quiet
#%pip install langchain-chroma --quiet
#%pip install -U langchain-community langgraph tavily-python langgraph-checkpoint-sqlite

In [1]:
# import getenv
import os
import getpass
import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()
# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI


In [2]:
import getpass
import os

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)

In [3]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! 😊 How can I assist you today?'

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Toronto")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in Toronto in May 2025 (Ontario)', 'url': 'https://world-weather.info/forecast/canada/toronto/may-2025/', 'content': 'Weather in Toronto in May 2025. Toronto Weather Forecast for May 2025 is ... Thursday, 8 May. +52°. Day. +55°. Overcast clouds. Friday, 9 May. +50°. Day. +55', 'score': 0.95598555}, {'title': 'Weather in Toronto', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Toronto', 'region': 'Ontario', 'country': 'Canada', 'lat': 43.6667, 'lon': -79.4167, 'tz_id': 'America/Toronto', 'localtime_epoch': 1746711461, 'localtime': '2025-05-08 09:37'}, 'current': {'last_updated_epoch': 1746711000, 'last_updated': '2025-05-08 09:30', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 8.7, 'wind_kph': 14.0, 'wind_degree': 23, 'wind_dir': 'NNE', 'pressure_mb': 1023.0, 'pressure_in': 30.21, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity'

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? 😊
ToolCalls: []


In [11]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Vancouver?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Vancouver'}, 'id': 'call_h4ySeJA7ZAW11lAaLBrfH8Nj', 'type': 'tool_call'}]


In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='7d77597a-2c2e-4bb3-a505-934550956561'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BUw1NfR0kY8NRy2skQXr8ZVXFgZ51', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop',

## Add streaming

In [14]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_8jn8AnvL8WhzR5YtWGRkdtFo)
 Call ID: call_8jn8AnvL8WhzR5YtWGRkdtFo
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1746712576, 'localtime': '2025-05-08 06:56'}, 'current': {'last_updated_epoch': 1746711900, 'last_updated': '2025-05-08 06:45', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/12

### add memory

In [15]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [16]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [17]:
# Use the agent
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi I am ARTURO! and i live in Oakville Ontario")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi I am ARTURO! and i live in Oakville Ontario
================================== Ai Message ==================================

Hello, Arturo! It's great to meet you. Oakville is such a beautiful place to live, with its stunning waterfront and great community vibes. What can I help you with today?


In [18]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_oGlJT4vfVIcFbt5BF6w8aWC7)
 Call ID: call_oGlJT4vfVIcFbt5BF6w8aWC7
  Args:
    query: current weather in Oakville Ontario
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in Oakville, Ontario", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Oakville', 'region': 'Ontario', 'country': 'Canada', 'lat': 43.4333, 'lon': -79.6667, 'tz_id': 'America/Toronto', 'localtime_epoch': 1746712629, 'localtime': '2025-05-08 09:57'}, 'current': {'last_updated_epoch': 1746711900, 'last_updated': '2025-05-08 09:45', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1